In [4]:
# !pip install spotipy

In [1]:
# Dependencies, building website, pulling data
import requests #works with url connection, makes python connect to internet
import json 
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
